In [14]:
!pip install -qU llama-index huggingface_hub llama-index-llms-huggingface-api llama-index-llms-ollama llama-index-readers-web

In [19]:
from IPython.display import Markdown, display

In [2]:
import os
from getpass import getpass
from huggingface_hub import login

/home/mahbub/Documents/dev/llama_index_projects/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
hf_token = getpass()

In [4]:
login(token=hf_token, add_to_git_credential=True)

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /home/mahbub/.cache/huggingface/token
Login successful


In [5]:
with open("best.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [6]:
text

'Despite its title this isn\'t meant to be the best essay. My goal here is to figure out what the best essay would be like.\n\nIt would be well-written, but you can write well about any topic. What made it special would be what it was about.\n\nObviously some topics would be better than others. It probably wouldn\'t be about this year\'s lipstick colors. But it wouldn\'t be vaporous talk about elevated themes either. A good essay has to be surprising. It has to tell people something they don\'t already know.\n\nThe best essay would be on the most important topic you could tell people something surprising about.\n\nThat may sound obvious, but it has some unexpected consequences. One is that science enters the picture like an elephant stepping into a rowboat. For example, Darwin first described the idea of natural selection in an essay written in 1844. Talk about an important topic you could tell people something surprising about. If that\'s the test of a great essay, this was surely the

In [7]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="tinyllama:latest", request_timeout=60.0, temperature=0)

llm

Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x71beb81f5f50>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x71be9c1dbec0>, completion_to_prompt=<function default_completion_to_prompt at 0x71be9c092700>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='tinyllama:latest', temperature=0.0, context_window=3900, request_timeout=60.0, prompt_key='prompt', json_mode=False, additional_kwargs={}, is_function_calling_model=True)

In [8]:
query = llm.complete("Why is the sky blue?")
query

CompletionResponse(text="The sky blue color is caused by the presence of various colors in the atmosphere, including blue light emitted from the sun and other sources such as volcanic ash, dust, and pollution. The blue hue is created when these different wavelengths of light combine to create a unique blend of colors that we perceive as blue. The exact composition of the sky's color depends on various factors such as atmospheric conditions, cloud cover, and sunlight intensity.", additional_kwargs={'tool_calls': []}, raw={'model': 'tinyllama:latest', 'created_at': '2024-08-18T22:05:48.366316201Z', 'message': {'role': 'assistant', 'content': "The sky blue color is caused by the presence of various colors in the atmosphere, including blue light emitted from the sun and other sources such as volcanic ash, dust, and pollution. The blue hue is created when these different wavelengths of light combine to create a unique blend of colors that we perceive as blue. The exact composition of the sk

In [9]:
from llama_index.core import PromptTemplate

qa_template = PromptTemplate(
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)

In [10]:
qa_template

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='We have provided context information below. \n---------------------\n{context_str}\n---------------------\nGiven this information, please answer the question: {query_str}\n')

In [11]:
question = "What is this essay about?"

prompt = qa_template.format(context_str=text, query_str=question)

response = llm.complete(prompt)

print(response.text)

The essay is about timelessness and how it can be achieved by writing about topics that readers already know but in much more detail than can be transmitted culturally. The author has written extensively on topics such as the importance of ambition, the tendency for young engineers to produce overcomplicated solutions, and the need for people to learn new things in order to surprise future generations. The author believes that writing about timelessness is an instance of breadth of applicability and that it can be achieved by applying to various fields and being very exacting with answers. Ultimately, the author's map of essay writing highlights the alternating stripes of inspiration and effort required for great essays, but also emphasizes the importance of persistence in achieving them.


In [12]:
question = "Who is the president of the United States?"

prompt = qa_template.format(context_str=text, query_str=question)

response = llm.complete(prompt)

print(response.text)

The given text does not provide any information about who the president of the United States is.


using SimpleWebPageReader

In [16]:
from llama_index.readers.web import SimpleWebPageReader

reader = SimpleWebPageReader(html_to_text=True).load_data(["https://www.ml.school/"])

In [18]:
reader[0]

Document(id_='https://www.ml.school/', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='> "This is the best machine learning course I\'ve done. Worth every cent."\n\n**Jose Reyes** , AI/ML at Cevo Australia\n\n# Building Machine Learning Systems That Don\'t Suck\n\nA **live, interactive program** that\'ll help you build **production-ready**\nmachine learning systems from the ground up.\n\nNext cohort: November 4 \\- 21, 2024\n\nCheck the schedule for more details about upcoming cohorts.\n\nI want to join!Sign in\n\n## Learn how to **design** , **build** , **deploy** , and **scale** machine\nlearning systems to solve real-world problems.\n\nI\'ll lose my mind if I see another book or course teaching people the same\nbasic ideas for the hundredth time. Most people are stuck in beginner mode,\nand finding help to solve real-world problems is hard.\n\nI want to change that.\n\nI started writing software 30 years ago. I\'ve

In [22]:
display(Markdown(reader[0].text))

> "This is the best machine learning course I've done. Worth every cent."

**Jose Reyes** , AI/ML at Cevo Australia

# Building Machine Learning Systems That Don't Suck

A **live, interactive program** that'll help you build **production-ready**
machine learning systems from the ground up.

Next cohort: November 4 \- 21, 2024

Check the schedule for more details about upcoming cohorts.

I want to join!Sign in

## Learn how to **design** , **build** , **deploy** , and **scale** machine
learning systems to solve real-world problems.

I'll lose my mind if I see another book or course teaching people the same
basic ideas for the hundredth time. Most people are stuck in beginner mode,
and finding help to solve real-world problems is hard.

I want to change that.

I started writing software 30 years ago. I've written pipelines and trained
models for some of the largest companies in the world. I want to show you how
to do the same.

This is the class I wish I had taken when I started.

This program will help you unlearn what you think machine learning is. It's a
practical, hands-on class where you'll learn from years of experience and
real-world examples.

When you join, you get **lifetime access** to the following:

  * 18 hours of live, interactive sessions. We'll use this time to discuss the first principles behind building machine learning systems.
  * 10 hours of step-by-step coding instructions. These practical sessions will show you how to build an end-to-end system from scratch.
  * A final project where you'll build a complete solution and receive direct feedback on your work.
  * 100 coding assignments and practice questions.
  * The entire source code of a working production system. It's yours. You can change and use it as you see fit.
  * A private community where you'll collaborate with thousands of people from different backgrounds.
  * Direct access to your instructor.
  * Lifetime access to every past and future cohort.
  * Program certificate upon completion.

And the best part is that you only pay once to join. There are no monthly
fees. No annual fees. No hidden costs. You pay once to join and benefit
forever until the end of time.

The program won't be easy. It'll take time and effort. But if you want to use
machine learning to solve real-world problems, **this is the class you don 't
want to miss.**

# Who Is This Program For?

## This is a practical, hands-on program for technical professionals who are
ready to put in the work.

This program is for **software engineers** , **data scientists** , **data
analysts** , **machine learning engineers** , **technical managers** , and
anyone anyone who wants to use machine learning to solve real-world problems.

Here are the criteria to succeed in the program:

  * You have experience writing code. We'll use Python throughout the class, but you won't have any problems if you know any other language.
  * You are familiar with basic machine learning terminology. This is not an introductory class. We'll move quickly over the basics to focus on the fundamental ideas that make systems work.
  * You are ready to put in the work to succeed.

> "I have learned a ton from Santiago in his class and it was actually what
> helped inspire me and get into the MLOps work that I'm doing now. Truly one
> of the most helpful online courses for doing real, full-scale machine
> learning."

![](/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fbrianhough.965e7d9d.jpeg&w=96&q=75)

Brian H. Hough

Software Engineer

# What Will You Learn?

## You'll come out with practical skills and insights into what it takes to
build systems that work in the real world.

Here is a summary of what makes this program unique:

  * You'll design and write the code to build an end-to-end machine learning system starting from scratch.
  * You'll learn best practices to tackle the most significant challenges machine learning engineers face to build, evaluate, run, monitor, and maintain machine learning systems in real-world scenarios.
  * You'll learn how to use techniques like active learning, distributed training, adversarial validation, human-in-the-loop deployments, model compression, test-time augmentation, testing in production, among many others.
  * You'll learn how to create training, deploying, monitoring, and inference pipelines using Amazon SageMaker and open-source tools.

Forget about theoretical concepts. This program will show you some of the
things I've learned from real-life examples I've built during more than 30
years in the industry.

Check the program syllabus →

  1. ![](/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fslides-thumbnail.270f2d17.jpg&w=3840&q=75)

### Real-life examples and case studies

Learn from practical experience building machine learning systems that work in
the real world.

  2. ![](/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fvideo-thumbnail.228cee07.jpg&w=3840&q=75)

### Live, interactive sessions

Ask questions and interact with the instructor and other students in real
time.

  3. ![](/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fcode-thumbnail.96685995.jpg&w=3840&q=75)

### Code walkthroughs

Step by step coding instructions to help you build a production system from
scratch.

# Upcoming Schedule

## Every iteration of the program gives you 18 hours of hands-on, live
training spread over 3 weeks.

Here are the upcoming cohorts:

  * Cohort 16: **November 4 - November 21, 2024. **1:00 PM EST

Live sessions will take place every Monday and Thursday at the same time. On
Wednesdays, we'll host office hours when you can bring your questions projects
or anything else you want to discuss.

  * **Monday** : Live session. 2 hours.
  * **Tuesday** : Individual work.
  * **Wednesday** : Optional office hours.
  * **Thursday** : Live session. 2 hours.
  * **Friday** : Individual work.

Do not wait for a specific cohort to join the program. You have lifetime
access to every past and future cohort, and the sooner you join, the more time
you have to prepare.

Every session is recorded. You can attend live or watch the recorded version
later.

> "This is one of the best classes I've ever purchased over the internet.
> Santiago is a terrific teacher. The ability he has to share knowledge is
> fantastic. I recommend this course. Worth 10x what he's charging."

![](/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fsaldistefano.d0b42e55.jpeg&w=96&q=75)

Sal DiStefano

# Ready To Join The Program?

## You'll get lifetime access. No monthly fees. No annual fees. No hidden
costs.

$500

### Pay once. Access forever.

Pay once to join the program and get lifetime access. You can participate in
as many iterations as you'd like. No restrictions.

  * Enjoy 18 hours of live, interactive sessions
  * Watch 10 hours of step-by-step coding instructions
  * Practice with 100 coding assignments
  * Access the complete source code of a production system
  * Learn how to start freelancing on Upwork
  * Get feedback and support from the community
  * Get direct feedback from your instructor

[Join now](https://buy.stripe.com/9AQcMP6Yjedy0WA4gg)

# Program Syllabus

## This program will teach you the practical skills and insights that will
help you build machine learning systems.

Here are the contents of the six live sessions of the program:

  1. ### Session 1 \- How To Start (Almost) Any Project

     * What makes production machine learning different from what you've learned.
     * The strategy to solve the right problem using the right solution.
     * Critical questions to ask before starting any project.
     * Problem framing, inversion, and the haystack principle for building successful applications.
     * The first rule of machine learning engineering and how to start building.
     * Data collection strategies. A technique to determine how much data you need.
     * The problem of selection bias and how to deal with it.
     * Labeling data. Human annotations, natural labels and weak supervision.
     * Active learning using the uncertainty and diversity sampling strategies.
  2. ### Session 2 \- How to Build a Model

     * The role of data cleaning and feature engineering to build better models.
     * Turning data into numbers using vectorization techniques.
     * Producing homogeneous features using normalization and standardization.
     * Handling and interpreting missing values using imputation techniques.
     * The approach to choosing the best model to solve any problem.
     * Random baselines and the zero-rule algorithm.
     * How to use overfitting to build models that don't suck.
     * Hyperparameter tuning and experiment tracking.
     * Measuring the quality of your holdout set.
     * An introduction to distributed training using data parallelism and model parallelism.

See the remaining sessions

> "This is an awesome course! This is my second round and I continue learning.
> I recommend it with complete confidence."

![](/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fjuanolano.ff7e749a.jpeg&w=96&q=75)

Juan Olano

Machine Learning Engineer

# Frequently Asked Questions

## If you can't find the answer to your question, please reach out on social
media and I'll be happy to help.

### How long will it take to complete the program?

If you are attending the live sessions, you should set aside a minimum of 4
hours every week during the three weeks of the program. This commitment will
be enough for engineer leaders or anyone not interested in the coding portion
of the program.

Those interested in implementing the concepts discussed in class should set
aside 2 to 4 hours weekly to complete the code walkthroughs and work on the
assignments.

### Are live sessions recorded?

Yes, we record every live session. You can decide when to attend classes live
or catch up asynchronously later using the recording.

### I'm a complete beginner. Is this program helpful for me?

This program is not an introduction to machine learning.

While we'll discuss many fundamental ideas behind machine learning, beginners
will find the sessions go much faster than what's optimal for them.

### What does "lifetime access" mean?

You only pay once to join the program and get immediate access to every past,
present, and future cohort.

Every new iteration of the program is better than the ones before. Many
students take classes once and then join a later cohort to benefit from the
updates.

The lifetime access removes any pressure from having to complete the program
when life gets in the way.

![](/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fheadshot.8b69bb12.jpg&w=3840&q=75)

# Hey! I'm Santiago.

## I'm the instructor of the program.

I'm a machine learning engineer with over two decades of experience building
and scaling enterprise software and machine learning systems.

I love neural networks. I love to make them work at scale.

From 2009 to 2023, I built products for Disney, Boston Dynamics, IBM, Dell,
G4S, Anheuser-Busch, and NextEra Energy, among other clients. I learned about
trade-offs and how to create products that work.

I started this program in March 2023. Since then, thousands of students have
graduated, and I can't wait to meet you in class.

[](https://twitter.com/svpino)[](https://linkedin.com/in/svpino)

Copyright © 2024 Tideily LLC

All rights reserved.

